In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
base_url = 'https://locations.dunkindonuts.com/en'

response = requests.get(url = base_url)

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')

In [8]:
dd_states = [item.get_text() for item in soup.select('.Directory-listLink')]

In [7]:
states = {
    'AK':'Alaska',
    'AL':'Alabama',
    'AR':'Arkansas',
    'AZ':'Arizona',
    'CA':'California',
    'CO':'Colorado',
    'CT':'Connecticut',
    'DC':'District of Columbia',
    'DE':'Delaware',
    'FL':'Florida',
    'GA':'Georgia',
    'HI':'Hawaii',
    'IA':'Iowa',
    'ID':'Idaho',
    'IL':'Illinois',
    'IN':'Indiana',
    'KS':'Kansas',
    'KY':'Kentucky',
    'LA':'Louisiana',
    'MA':'Massachusetts',
    'MD':'Maryland',
    'ME':'Maine',
    'MI':'Michigan',
    'MN':'Minnesota',
    'MO':'Missouri',
    'MS':'Mississippi',
    'MT':'Montana',
    'NC':'North Carolina',
    'ND':'North Dakota',
    'NE':'Nebraska',
    'NH':'New Hampshire',
    'NJ':'New Jersey',
    'NM':'New Mexico',
    'NV':'Nevada',
    'NY':'New York',
    'OH':'Ohio',
    'OK':'Oklahoma',
    'OR':'Oregon',
    'PA':'Pennsylvania',
    'RI':'Rhode Island',
    'SC':'South Carolina',
    'SD':'South Dakota',
    'TN':'Tennessee',
    'TX':'Texas',
    'UT':'Utah',
    'VA':'Virginia',
    'VT':'Vermont',
    'WA':'Washington',
    'WI':'Wisconsin',
    'WV':'West Virginia',
    'WY':'Wyoming'
}


In [19]:
def get_key(val):
    for key, value in states.items():
        if val == value:
            return key
        if val == "Washington DC": # special case because DD website does not list DC as "District of Columbia"
            return "DC"
    return "key doesn't exist"

In [26]:
dd_state_abvs = [get_key(val).lower() for val in dd_states]

In [27]:
base_url + "/" + dd_state_abvs[0]

'https://locations.dunkindonuts.com/en/al'

In [28]:
page = requests.get(url = base_url + "/" + dd_state_abvs[0])
state_soup = BeautifulSoup(page.content, 'html.parser')

In [34]:
state_twns = [item.get_text() for item in state_soup.select('.Directory-listItem')]

In [35]:
page = requests.get(url = 'https://locations.dunkindonuts.com/en/al/alabaster')
new_soup = BeautifulSoup(page.content, 'html.parser')

In [49]:
main_page = new_soup.find(id = 'main')

In [51]:
teaser = main_page.find_all(class_ = 'Teaser Teaser--directory')

In [52]:
address = teaser[0]

In [53]:
print(address.prettify())

<div class="Teaser Teaser--directory">
 <a class="Teaser-name" href="../../en/al/alabaster/1114-1st-st-n/358766">
  1114 1st St N
 </a>
 <div class="Teaser-hours">
  <span class="c-hours-today js-hours-today" data-days='[{"day":"MONDAY","intervals":[{"end":2000,"start":500}],"isClosed":false},{"day":"TUESDAY","intervals":[{"end":2000,"start":500}],"isClosed":false},{"day":"WEDNESDAY","intervals":[{"end":2000,"start":500}],"isClosed":false},{"day":"THURSDAY","intervals":[{"end":2000,"start":500}],"isClosed":false},{"day":"FRIDAY","intervals":[{"end":2000,"start":500}],"isClosed":false},{"day":"SATURDAY","intervals":[{"end":2000,"start":600}],"isClosed":false},{"day":"SUNDAY","intervals":[{"end":2000,"start":600}],"isClosed":false}]' data-utc-offsets='[{"offset":-18000,"start":1647158400},{"offset":-21600,"start":1667718000},{"offset":-18000,"start":1678608000}]'>
   <span class="c-hours-today-details-row js-day-of-week-row" data-day-of-week-end-index="1" data-day-of-week-start-index="1"

In [59]:
closer = address.find(class_ = 'Teaser-name')

In [73]:
store_address = list(closer.attrs.values())[0]

In [74]:
store_address = store_address[8:]

In [76]:
base_url + store_address

'https://locations.dunkindonuts.com/en/al/alabaster/1114-1st-st-n/358766'

In [77]:
store_page = requests.get(url = base_url + store_address)

In [79]:
store_soup = BeautifulSoup(store_page.content, 'html.parser')

In [84]:
address_data = [item.get_text() for item in\
    store_soup.select('.c-address-postal-code , .c-address-city , .c-address-state , .c-address-street-2 , .c-address-street-1')]

In [109]:
phone_data = store_soup.select('#phone-main')[0].get_text()

In [106]:
hrs_data = [item.get_text() for item in \
    store_soup.select('.c-hours-details-row-intervals')]

In [86]:
field_names = [
    'address_line_1', 
    'address_line_2', 
    'city', 
    'state', 
    'zip',
    'phone',
    'mon_hrs',
    'tue_hrs',
    'wed_hrs',
    'thu_hrs',
    'fri_hrs',
    'sat_hrs',
    'sun_hrs']

In [122]:
feature_names = [
    'Drive Thru',
    'On-the-Go Mobile Ordering',
    "Accepts Dunkin' Cards",
    'K-Cup Pods',
    'Curbside Pick Up',
    'Baskin-Robbins'
]

In [87]:
address_dict = dict(zip(field_names, address_data))

In [117]:
store_24_page = requests.get(url = 'https://locations.dunkindonuts.com/en/ny/adams/48-west-church-st/346213')
store_24_soup = BeautifulSoup(store_24_page.content, 'html.parser')

In [121]:
for item in store_24_soup.select('.Core-features'):
    print(item.get_text())

FeaturesDrive ThruOn-the-Go Mobile OrderingAccepts Dunkin' CardsK-Cup Pods


In [119]:
no_feats = [item.get_text() for item in store_24_soup.select('Core.features')]

In [120]:
print(type(no_feats))

<class 'list'>
